In [1]:
import requests

import gtfs_realtime_NYCT_pb2
import gtfs_realtime_pb2
import polars as pl
from polars import col
import re
from PIL import Image
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import pandas as pd
from PIL import Image
import pyarrow
import json
import bisect

# import betterproto

In [2]:
import plotly.io as pio

# Load the plot from an HTML file
with open("map_plot.json", "r") as f:
    fig_json = json.load(f)

fig = go.Figure(fig_json)

In [3]:
stops = pl.read_csv(
    "stops.txt",
    separator=",",
    has_header=True,
    schema_overrides={"parent_station": pl.String},
)

shapes = pl.read_csv(
    "shapes.txt",
    separator=",",
    has_header=True,
)

colors = pl.read_csv("MTA_Colors_20240623.csv", separator=",", has_header=True)

In [4]:
colors = colors.filter(col("Operator") == "New York City Subway")
colors = colors.with_columns(
    col("Service").str.split(",")
)  # Split the comma-delimited values into lists
colors = colors.explode("Service")  # Explode the lists into separate rows

In [5]:
stop_status = {"0": "Incoming At", "1": "Stopped At", "2": "In Transit To"}

In [6]:
api_endpoints = {
    "ACE": r"https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs-ace",
    "BDFM": r"https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs-bdfm",
    "G": r"https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs-g",
    "JZ": r"https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs-jz",
    "NQRW": r"https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs-nqrw",
    "L": r"https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs-l",
    "1234567": r"https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs",
    "SI": r"https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs-si",
}

In [7]:
response = requests.get(
    "https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs-bdfm"
)

In [35]:
response = requests.get(api_endpoints["1234567"])
feed = gtfs_realtime_pb2.FeedMessage()
feed.ParseFromString(response.content)

148629

In [9]:
feed = gtfs_realtime_pb2.FeedMessage()

In [10]:
feed.ParseFromString(response.content)

152982

In [11]:
feed.entity[0].trip_update.trip.route_id

'1'

In [12]:
shape_unpack_re = re.compile(r"^(\w{1}).*\.+(\w+)$")


def shape_unpack(shape):
    m = re.match(shape_unpack_re, shape)
    return m.group(1), m.group(2)

In [13]:
shapes_clean = shapes.with_columns(
    [
        shapes["shape_id"]
        .map_elements(lambda x: shape_unpack(x)[0], return_dtype=str)
        .alias("Line"),
        shapes["shape_id"]
        .map_elements(lambda x: shape_unpack(x)[1], return_dtype=str)
        .alias("Line_Variation"),
    ]
)

In [14]:
colors

Operator,Service,Hex color,CMYK
str,str,str,str
"""New York City Subway""","""A""","""#0039A6""","""(100,56,0,0)"""
"""New York City Subway""","""C""","""#0039A6""","""(100,56,0,0)"""
"""New York City Subway""","""E""","""#0039A6""","""(100,56,0,0)"""
"""New York City Subway""","""B""","""#FF6319""","""(0,60,100,0)"""
"""New York City Subway""","""D""","""#FF6319""","""(0,60,100,0)"""
…,…,…,…
"""New York City Subway""","""3""","""#EE352E""","""(0,91,76,0)"""
"""New York City Subway""","""4""","""#00933C""","""(100,0,91,6)"""
"""New York City Subway""","""5""","""#00933C""","""(100,0,91,6)"""


In [15]:
stops_clean["Hex color"].value_counts()

NameError: name 'stops_clean' is not defined

In [16]:
stop_removal_re = r".*[NS]$"

stops = stops.filter(~stops["stop_id"].str.contains(stop_removal_re))

In [17]:
stop_unpack_re = re.compile(r"^(\w{1})(\d{2})")


def stop_unpack(stop):
    m = re.match(stop_unpack_re, stop)
    return m.group(1), m.group(2)

In [18]:
stops_clean = stops[["stop_id", "stop_name", "stop_lat", "stop_lon"]].with_columns(
    [
        stops["stop_id"]
        .map_elements(lambda x: stop_unpack(x)[0], return_dtype=str)
        .alias("Line"),
        stops["stop_id"]
        .map_elements(lambda x: stop_unpack(x)[1], return_dtype=str)
        .alias("Order"),
    ]
)

In [19]:
# stops_clean = stops_clean.join(line_points, left_on="Line", right_on="Line", how="left")
stops_clean = (
    stops_clean.join(colors, left_on="Line", right_on="Service", how="left")
    .with_columns(pl.col("Hex color").fill_null("#858585"))
    .select(
        ["stop_name", "stop_id", "stop_lat", "stop_lon", "Line", "Order", "Hex color"]
    )
)

/var/folders/7y/62_9jkxd1k70c309j3h2bgbh0000gn/T/ipykernel_62032/506250869.py:3: DeprecationWarning:

The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.



In [20]:
shapes_final = shapes_clean.join(
    stops_clean.select(["stop_lon", "stop_lat", "stop_name", "stop_id"]),
    left_on=("shape_pt_lon", "shape_pt_lat"),
    right_on=("stop_lon", "stop_lat"),
    how="left",
)

/var/folders/7y/62_9jkxd1k70c309j3h2bgbh0000gn/T/ipykernel_62032/1807151076.py:1: DeprecationWarning:

The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.



In [21]:
shapes_final

shape_id,shape_pt_sequence,shape_pt_lat,shape_pt_lon,Line,Line_Variation,stop_name,stop_id
str,i64,f64,f64,str,str,str,str
"""1..N03R""",0,40.702068,-74.013664,"""1""","""N03R""","""South Ferry""","""142"""
"""1..N03R""",1,40.703199,-74.014792,"""1""","""N03R""",null,null
"""1..N03R""",2,40.703226,-74.01482,"""1""","""N03R""",null,null
"""1..N03R""",3,40.703253,-74.014846,"""1""","""N03R""",null,null
"""1..N03R""",4,40.70328,-74.01487,"""1""","""N03R""",null,null
…,…,…,…,…,…,…,…
"""SI.S31R""",685,40.513696,-74.250493,"""S""","""S31R""",null,null
"""SI.S31R""",686,40.513579,-74.250706,"""S""","""S31R""",null,null
"""SI.S31R""",687,40.513458,-74.250917,"""S""","""S31R""",null,null


In [22]:
train_route = shapes_final.filter(pl.col("shape_id") == "4..S01R")

In [23]:
prev_stop = train_route.select(
    [
        pl.arg_where(pl.col("stop_id") == "142"),
    ]
)[0, 0]

next_stop = train_route.select(
    [
        pl.arg_where(pl.col("stop_id") == "139"),
    ]
)[0, 0]

In [24]:
in_route = train_route[prev_stop : next_stop + 1]

TypeError: slice indices must be integers or None or have an __index__ method

In [25]:
import math


def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in kilometers
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(math.radians(lat1))
        * math.cos(math.radians(lat2))
        * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return distance

In [193]:
# Define a function to calculate distance using lead coordinates
def calculate_distance_within_line(df):

    df = df.sort("shape_pt_sequence")

    # Create lead columns for lat and lon
    df = df.with_columns(
        [
            df["shape_pt_lat"].shift(1).alias("lag_lat"),
            df["shape_pt_lon"].shift(1).alias("lag_lon"),
        ]
    )

    # Apply the Haversine function to each row and add the result as a new column
    return df.with_columns(
        [
            pl.concat_list(["shape_pt_lat", "shape_pt_lon", "lag_lat", "lag_lon"])
            .map_elements(
                lambda row: (
                    haversine(row[0], row[1], row[2], row[3])
                    if row[2] is not None and row[3] is not None
                    else None
                ),
                return_dtype=pl.Float64,
            )
            .alias("distance")
        ]
    )


# Group by 'Line', sort each group by 'line_order', and apply the distance calculation
result = in_route.group_by("Line", maintain_order=True).map_groups(
    calculate_distance_within_line
)


sums = result.select(pl.col("distance").drop_nulls()).to_series().sum()
result = result.with_columns((result["distance"] / sums).alias("proportion"))
result = result.with_columns(pl.col("proportion").cum_sum().round(7).alias("cum_sum"))

In [186]:
result

shape_id,shape_pt_sequence,shape_pt_lat,shape_pt_lon,Line,Line_Variation,stop_name,stop_id,lead_lat,lead_lon,distance,proportion,cum_sum
str,i64,f64,f64,str,str,str,str,f64,f64,f64,f64,f64
"""1..N03R""",0,40.702068,-74.013664,"""1""","""N03R""","""South Ferry""","""142""",null,null,null,null,null
"""1..N03R""",1,40.703199,-74.014792,"""1""","""N03R""",null,null,40.702068,-74.013664,0.157663,0.238643,0.2386431
"""1..N03R""",2,40.703226,-74.01482,"""1""","""N03R""",null,null,40.703199,-74.014792,0.003819,0.005781,0.2444236
"""1..N03R""",3,40.703253,-74.014846,"""1""","""N03R""",null,null,40.703226,-74.01482,0.003717,0.005626,0.25005
"""1..N03R""",4,40.70328,-74.01487,"""1""","""N03R""",null,null,40.703253,-74.014846,0.00362,0.00548,0.2555298
…,…,…,…,…,…,…,…,…,…,…,…,…
"""1..N03R""",30,40.704514,-74.015076,"""1""","""N03R""",null,null,40.704487,-74.015086,0.003118,0.00472,0.4689643
"""1..N03R""",31,40.704541,-74.015065,"""1""","""N03R""",null,null,40.704514,-74.015076,0.003142,0.004756,0.4737204
"""1..N03R""",32,40.704568,-74.015052,"""1""","""N03R""",null,null,40.704541,-74.015065,0.003196,0.004838,0.478558


In [194]:
def linear_distance(lon1, lat1, lon2, lat2, fraction):
    lat = lat1 + (lat2 - lat1) * fraction
    lon = lon1 + (lon2 - lon1) * fraction
    return lat, lon

In [219]:
def calculate_position(api_time, departure, arrival, df: pl.DataFrame):
    trip_time = arrival - departure
    since_departure = api_time - departure
    proportion_traveled = since_departure / trip_time
    # TODO: Calculate proportions in case a train is skipping a stop

    loc = bisect.bisect_left(
        (cum_sum := df["cum_sum"].fill_null(0).to_list()), proportion_traveled
    )
    if proportion_traveled in cum_sum:
        return df[loc, :].select(["shape_pt_lat", "shape_pt_lon"]).row(0)
    else:
        temp = df[loc, :][0].to_dict(as_series=False)
        return linear_distance(
            temp["lag_lon"][0],
            temp["lag_lat"][0],
            temp["shape_pt_lon"][0],
            temp["shape_pt_lat"][0],
            (proportion_traveled - (cum_sum[loc - 1] if loc != 1 else 0))
            / (cum_sum[loc] - cum_sum[loc - 1]),
        )

In [ ]:
ent

In [211]:
result[33, "cum_sum"]

0.5711883

In [212]:
calculate_position(1720729868, 1720729808, 1720729888, result)

(40.7055249031257, -74.0146427841501)

In [220]:
coords = [
    calculate_position(x, 1720729828, 1720729888, result)
    for x in range(1720729828, 1720729889)
]

In [221]:
import plotly.graph_objects as go

# Example list of coordinates (latitude, longitude)


# Separate the list of tuples into two lists: latitudes and longitudes
latitudes, longitudes = zip(*coords)

# Create a scatter plot on a map
fig = go.Figure(
    go.Scattermapbox(
        mode="markers+lines",
        lon=longitudes,
        lat=latitudes,
        marker={"size": 10},
        line=dict(width=2, color="blue"),
    )
)

# Set the layout of the map
fig.update_layout(
    mapbox={
        "style": "open-street-map",
        "center": {
            "lat": sum(latitudes) / len(latitudes),
            "lon": sum(longitudes) / len(longitudes),
        },
        "zoom": 10,
    },
    showlegend=False,
)

# Show the plot
fig.show()

In [37]:
[
    x
    for x in feed.entity
    if any(
        [
            x.vehicle.trip.trip_id == "109300_2..N01R",
            x.trip_update.trip.trip_id == "109300_2..N01R",
        ]
    )
]

[id: "000082"
 trip_update {
   trip {
     trip_id: "109300_2..N01R"
     start_date: "20240711"
     route_id: "2"
     [transit_realtime.nyct_trip_descriptor] {
       train_id: "02 1813  FLA/241"
       is_assigned: true
     }
   }
   stop_time_update {
     arrival {
       time: 1720741158
     }
     departure {
       time: 1720741158
     }
     stop_id: "210N"
     [transit_realtime.nyct_stop_time_update] {
       scheduled_track: "3"
       actual_track: "3"
     }
   }
   stop_time_update {
     arrival {
       time: 1720741248
     }
     departure {
       time: 1720741248
     }
     stop_id: "209N"
     [transit_realtime.nyct_stop_time_update] {
       scheduled_track: "3"
     }
   }
   stop_time_update {
     arrival {
       time: 1720741368
     }
     departure {
       time: 1720741368
     }
     stop_id: "208N"
     [transit_realtime.nyct_stop_time_update] {
       scheduled_track: "3"
     }
   }
   stop_time_update {
     arrival {
       time: 1720741458
  

In [36]:
train_4 = [
    x
    for x in feed.entity
    if any(
        [
            x.vehicle.trip.trip_id == "126900_4..N06R",
            x.trip_update.trip.trip_id == "126900_4..N06R",
        ]
    )
]

In [37]:
train_4

[]

In [61]:
test = [
    x.trip_update.stop_time_update
    for x in feed.entity
    if x.trip_update.trip.trip_id == "131550_2..N01R"
]
test

[[arrival {
   time: 1720750757
 }
 departure {
   time: 1720750757
 }
 stop_id: "233N"
 [transit_realtime.nyct_stop_time_update] {
   scheduled_track: "4"
   actual_track: "4"
 }
 , arrival {
   time: 1720750847
 }
 departure {
   time: 1720750847
 }
 stop_id: "232N"
 [transit_realtime.nyct_stop_time_update] {
   scheduled_track: "3"
 }
 , arrival {
   time: 1720750937
 }
 departure {
   time: 1720750937
 }
 stop_id: "231N"
 [transit_realtime.nyct_stop_time_update] {
   scheduled_track: "3"
 }
 , arrival {
   time: 1720751147
 }
 departure {
   time: 1720751147
 }
 stop_id: "230N"
 [transit_realtime.nyct_stop_time_update] {
   scheduled_track: "3"
 }
 , arrival {
   time: 1720751207
 }
 departure {
   time: 1720751207
 }
 stop_id: "229N"
 [transit_realtime.nyct_stop_time_update] {
   scheduled_track: "3"
 }
 , arrival {
   time: 1720751357
 }
 departure {
   time: 1720751357
 }
 stop_id: "228N"
 [transit_realtime.nyct_stop_time_update] {
   scheduled_track: "3"
 }
 , arrival {
   time

In [64]:
test[0][0].departure.time

1720750757

In [68]:
trains_tracked = {}

In [69]:
def departure_time(updates):
    try:
        return updates[0][0].departure.time
    except IndexError:
        return None

In [70]:
for trip_id in set(
    [x.vehicle.trip.trip_id for x in feed.entity if x.vehicle.trip.trip_id]
):
    vehicle = [
        x.vehicle
        for x in feed.entity
        if x.HasField("vehicle") and x.vehicle.trip.trip_id == trip_id
    ]
    updates = [
        x.trip_update.stop_time_update
        for x in feed.entity
        if x.HasField("trip_update") and x.trip_update.trip.trip_id == trip_id
    ]
    trains_tracked[trip_id] = {
        "prev_departure_time": departure_time(updates),
        "current_schedule": updates,
        "current_loc_info": vehicle,
    }

In [71]:
trains_tracked

{'133150_6..N01R': {'prev_departure_time': 1720750852,
  'current_schedule': [[arrival {
     time: 1720750852
   }
   departure {
     time: 1720750852
   }
   stop_id: "634N"
   [transit_realtime.nyct_stop_time_update] {
     scheduled_track: "4"
   }
   , arrival {
     time: 1720750912
   }
   departure {
     time: 1720750912
   }
   stop_id: "633N"
   [transit_realtime.nyct_stop_time_update] {
     scheduled_track: "4"
   }
   , arrival {
     time: 1720750972
   }
   departure {
     time: 1720750972
   }
   stop_id: "632N"
   [transit_realtime.nyct_stop_time_update] {
     scheduled_track: "4"
   }
   , arrival {
     time: 1720751092
   }
   departure {
     time: 1720751092
   }
   stop_id: "631N"
   [transit_realtime.nyct_stop_time_update] {
     scheduled_track: "4"
   }
   , arrival {
     time: 1720751182
   }
   departure {
     time: 1720751182
   }
   stop_id: "630N"
   [transit_realtime.nyct_stop_time_update] {
     scheduled_track: "4"
   }
   , arrival {
     time: 

In [240]:
while True:
    prev = train_4[0].trip_update.stop_time_update[0]

arrival {
  time: 1720748619
}
departure {
  time: 1720748619
}
stop_id: "626S"
[transit_realtime.nyct_stop_time_update] {
  scheduled_track: "2"
}

# Plotting Train

In [ ]:
fig.add_trace(
    go.Scattermapbox(
        mode="markers",
        name=d["Line"][0],
        lon=d["stop_lon"],
        lat=d["stop_lat"],
        text=d["stop_name"],
        marker={"size": 8},
        line={"width": 4, "color": "#EE352E"},
    )
)